In [1]:
%load_ext autoreload
import xml.dom.minidom
# import ete3
# import pandas as pd
# import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import xml.etree.ElementTree as ET
# import xmlschema

In [2]:
%autoreload
from maxes.xes_loader2 import XesLoader, XesLog
# from maxes.xes_file import XesFile
# from maxes.print_xes_info import PrintXesInfo
import maxes.analyze_sequence
import maxes.serialization.serialize
# import maxes.graphs
# import maxes.utils
from maxes.generators.xes_generator1 import XesGenerator1

In [3]:
file_path = "../data/Activities of daily living of several individuals_1_all/data/activitylog_uci_detailed_labour.xes/activitylog_uci_detailed_labour.xes"

loader = XesLoader({ "drop_timezones": True })
log = loader.load(file_path)
loader.errors

[]

In [4]:
print(len(log.traces))
print(*[(len(trace.events), len(trace.attributes))
        for trace in log.traces[:5]], sep="\n")

25
(92, 2)
(62, 2)
(64, 2)
(62, 2)
(58, 2)


In [5]:
print(*[child.tag for child in log.loader.xml_log_skeleton.getroot()], sep="\n")

{http://www.xes-standard.org}extension
{http://www.xes-standard.org}extension
{http://www.xes-standard.org}extension
{http://www.xes-standard.org}global
{http://www.xes-standard.org}global
{http://www.xes-standard.org}classifier
{http://www.xes-standard.org}string
{http://www.xes-standard.org}string
{http://www.xes-standard.org}string


In [6]:
for trace in log.traces:
    trace.update_df()

In [32]:
log.traces[0].events[0].time_timestamp()

datetime.datetime(2012, 11, 12, 0, 48, 38, tzinfo=tzoffset(None, 3600))

In [71]:
generator = XesGenerator1().fit(log)
generated_log = generator.generate()
generated_log.traces[0].update_df()
generated_log.traces[0].df

,concept:name,lifecycle:transition,Column_4,time:timestamp
0,Start,start,sleeping,2012-11-16 01:48:31.599171
1,Start,complete,watchingtv,2012-11-16 01:48:31.035940
2,washing,start,eatingBreakfast,2012-11-16 01:48:30.838434
3,washing,complete,sleeping,2012-11-16 01:50:05.697161
4,grooming,start,watchingtv,2012-11-16 01:51:55.403235
...,...,...,...,...
57,outdoors,complete,prepareLunch,2012-11-16 16:44:28.730189
58,watchingtv,start,washing,2012-11-16 16:47:10.572521
59,watchingtv,complete,prepareBreakfast,2012-11-16 17:58:19.924550
60,End,start,toilet,2012-11-16 17:58:21.219076


In [69]:
generated_log.traces[0].events[0].time_timestamp.__class__

list

In [56]:
from sklearn.neighbors import KernelDensity
kde = KernelDensity().fit([[i] for i in [1, 2, 3, 4 ,5]])
kde.sample()

array([[3.66828596]])

In [45]:
log.traces[0].events[0].time_timestamp.

datetime.datetime(2012, 11, 12, 0, 48, 38, tzinfo=tzoffset(None, 3600))

In [39]:
# fig, ax = plt.subplots()
# fig.set_size_inches(20, 20)
# fig.dpi = 150
# maxes.analyze_sequence.draw_nx_graph(generator.sequence_graph_, edge_label="frequency", ax=ax, pos=nx.kamada_kawai_layout(generator.sequence_graph_))

In [74]:
generated_log_ET = maxes.serialization.serialize.Serializer().serialize(generated_log, xml_log_skeleton=log.loader.xml_log_skeleton)
ET.indent(generated_log_ET)
ET.register_namespace("", "http://www.xes-standard.org")
with open("C:\\vt\\md\\maxes\\output\\generated1.xml","w") as file:
    generated_log_ET.write(file, encoding="unicode")

In [70]:
original_xml_log = log.loader.tree.getroot()
new_xml_log = ET.Element(original_xml_log.tag, original_xml_log.attrib)
print(new_xml_log.tag)
ET.register_namespace("", "http://www.xes-standard.org")
new_tree = ET.ElementTree(new_xml_log)
print(ET.tostring(new_xml_log, encoding="unicode"))

{http://www.xes-standard.org}log
<log xmlns="http://www.xes-standard.org" xes.version="1.0" xes.creator="Fluxicon Disco" />


In [3]:
def generate_xes_for_file(source_file_path, destination_file_path):
    loader = XesLoader({ "drop_timezones": True })

    print("Loading")
    log = loader.load(source_file_path)

    if len(loader.errors):
        raise RuntimeError("Errors while loading XES")

    print("Creating dataframes")
    for trace in log.traces:
        trace.update_df()

    print("Fitting")
    generator = XesGenerator1(debug=True).fit(log)

    print("Generating")
    generated_log = generator.generate()

    print("Serializing")
    generated_log_ET = maxes.serialization.serialize.Serializer().serialize(generated_log, xml_log_skeleton=log.loader.xml_log_skeleton)

    print("Formatting XML")
    ET.indent(generated_log_ET)

    print("Writing XML")
    ET.register_namespace("", "http://www.xes-standard.org")
    with open(destination_file_path, "w") as file:
        generated_log_ET.write(file, encoding="unicode")


In [4]:
# generate_xes_for_file(
#     source_file_path="../data/Activities of daily living of several individuals_1_all/data/activitylog_uci_detailed_labour.xes/activitylog_uci_detailed_labour.xes",
#     destination_file_path="C:\\vt\\md\\maxes\\output\\generated1.xml"
# )
generate_xes_for_file(
    source_file_path="C:\\vt\\md\\maxes\\data\\BPI Challenge 2012_1_all\\BPI_Challenge_2012.xes\\BPI_Challenge_2012.xes",
    destination_file_path="C:\\vt\\md\\maxes\\output\\generated2.xml"
)

Loading
Creating dataframes
Fitting
Validating
Analyzing sequence graph
Gathering meta data
Fitting models for "time:timestamp" attribute
Fitting models for other attributes
Generating
Serializing
Formatting XML
Writing XML
